In [1]:
import numpy as np

In [2]:
np.random.seed(0)

In [3]:
data = "hello world"
chars = list(set(data))
vocab_size = len(chars)

chars_to_indices = {char: i for i, char in enumerate(chars)}
indices_to_chars = {i: char for i, char in enumerate(chars)}

chars_to_indices, indices_to_chars

({'r': 0, 'd': 1, 'o': 2, 'e': 3, 'h': 4, 'l': 5, ' ': 6, 'w': 7},
 {0: 'r', 1: 'd', 2: 'o', 3: 'e', 4: 'h', 5: 'l', 6: ' ', 7: 'w'})

In [4]:
# RNN configs
embedding_size = vocab_size
hidden_size = 64
output_size = vocab_size

In [5]:
# Matrices
Wxh = np.random.randn(embedding_size, hidden_size)
Whh = np.random.randn(hidden_size, hidden_size)
bh = np.random.randn(1, hidden_size)

Why = np.random.randn(hidden_size, output_size)
by = np.random.randn(1, output_size)

In [6]:
def softmax(z):
    e_z = np.exp(z)
    return e_z / e_z.sum(axis=1)

def rnn(x_t, h_prev):
    assert x_t.shape == (1, embedding_size)
    assert h_prev.shape == (1, hidden_size)

    h_next = np.tanh(x_t @ Wxh + h_prev @ Whh + bh)
    probs = softmax(h_next @ Why + by)
    
    return probs, h_next

def sample(char, n):
    # sample rnn n times starting with the first char
    x = np.zeros((1, vocab_size))
    idx = chars_to_indices[char]
    x[:, idx] = 1 # one hot encoding
    h =  np.zeros((1, hidden_size)) # hidden state
    idxes = []
    
    for i in range(n):
        probs, h = rnn(x, h) # xt, hprev -> rnn -> probs, hnext
        idx = np.random.choice(vocab_size, p=probs.ravel()) # .ravel returns a 1d array
        
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        idxes.append(idx)

    chars = "".join([indices_to_chars[i] for i in idxes])
    return chars

In [7]:
x_t = np.random.randn(1, embedding_size)
h_prev = np.random.randn(1, hidden_size)

probs, h_next = rnn(x_t, h_prev)
probs.shape, h_next.shape

((1, 8), (1, 64))

In [8]:
inp = [chars_to_indices[c] for c in data[:-1]]
target = [chars_to_indices[c] for c in data[1:]]

inp, target

([4, 3, 5, 5, 2, 6, 7, 2, 0, 5], [3, 5, 5, 2, 6, 7, 2, 0, 5, 1])

In [9]:
sample("h", 4)

'edd '

In [14]:
lr = 1e-2
for e in range(100):
    # compute loss
    loss = 0
    xs, ps, hs = {}, {}, {}
    h = np.zeros((1, hidden_size))
    hs[-1] = h
    for t in range(len(inp)):
        idx = inp[t]
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        probs, h = rnn(x, h)
        xs[t] = x
        ps[t] = probs # save probs (we'll use this in backprop)
        hs[t] = h

        # cross entropy
        pred = probs[0, target[t]]
        loss += -np.log(pred)

    # backprop, calculate gradients
    dL_dWxh = np.zeros_like(Wxh)
    dL_dWhh = np.zeros_like(Whh)
    dL_dbh = np.zeros_like(bh)
    dL_Why = np.zeros_like(Why)
    dL_dby = np.zeros_like(by)
    dF_dh = np.zeros((1, hidden_size))

    for t in reversed(range(len(inp))):
        dL_dz2 = np.copy(ps[t])
        target_idx = target[t]
        dL_dz2[:, target_idx] -= 1

        # 2nd layer
        dL_Why += dL_dz2 * hs[t].T
        dL_dby += dL_dz2

        # 1st layer
        dh_dz1 = 1 - hs[t] ** 2
        dL_dh = (dL_dz2 @ Why.T) + dF_dh 

        dL_dWxh += dL_dh * dh_dz1 * xs[t].T
        dL_dWhh += dL_dh * dh_dz1 * hs[t - 1].T
        dL_dbh += dL_dh * dh_dz1

        dF_dh = dL_dh * (dh_dz1 @ Whh.T)

    # clip gradients
    for gradient in [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]:
        np.clip(gradient, -1, 1, out=gradient)

    # gradient descent
    for weights, gradient in zip([Wxh, Whh, Why, bh, by], [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]):
        weights -= lr * gradient

    # break
    
    print(f"[Loss]: {loss}")

[Loss]: 0.020765789079127087
[Loss]: 0.020633137402769716
[Loss]: 0.020500878317168434
[Loss]: 0.02036723471958046
[Loss]: 0.020232051732413662
[Loss]: 0.02009778443933907
[Loss]: 0.01996898806849871
[Loss]: 0.019850844867482648
[Loss]: 0.0197486222151627
[Loss]: 0.01966910852283978
[Loss]: 0.019624543393132556
[Loss]: 0.019642560847256282
[Loss]: 0.019800126334480004
[Loss]: 0.02039600607446793
[Loss]: 0.02357711900876883
[Loss]: 0.036444903634356304
[Loss]: 0.465839406252212
[Loss]: 1.8801207803575928
[Loss]: 0.1057172080488387
[Loss]: 0.06114307402197965
[Loss]: 0.2663070048964097
[Loss]: 0.6478268728512007
[Loss]: 0.38411541602641647
[Loss]: 0.08797001303292297
[Loss]: 0.11876099617945426
[Loss]: 0.09592841563444966
[Loss]: 0.06305123224259446
[Loss]: 0.06053883861258935
[Loss]: 0.05890166015433987
[Loss]: 0.058856103924463735
[Loss]: 0.05988279824732445
[Loss]: 0.06518095580807341
[Loss]: 0.0957360772238684
[Loss]: 0.21725592329187926
[Loss]: 1.0893957217008068
[Loss]: 0.017773844

In [13]:
sample("h", 10)

'ello world'